# Phase 1–3 Demo: Shared Kernel + Objectives + Leiden+


This notebook demonstrates:

1. Building a small weighted undirected graph
2. Running Leiden via the **shared kernel** (Phase 1)
3. Plugging in **ObjectivePort** implementations (Phase 2)
4. Performing **resolution scans + consensus** (Leiden+) (Phase 3)

> Assumes this notebook is opened from your repo (so that `src/` is available).


In [ ]:

# --- Repo path bootstrap ---
import sys, os
from pathlib import Path
p = Path.cwd()
for _ in range(8):
    if (p / "src").exists():
        sys.path.insert(0, str(p))
        break
    p = p.parent
print("Using repo root:", p)


## 1) Build an example graph

In [ ]:

from src.lib.adapter.graph.from_edges import EdgesGraphBuilder

edges = [
    (1, 2, 1.0), (2, 3, 1.0), (3, 1, 1.0),   # triangle
    (4, 5, 1.0), (5, 6, 1.0), (6, 4, 1.0),   # triangle
    (3, 4, 0.1),                              # weak bridge
]
G = EdgesGraphBuilder().build(edges)
print("Nodes:", len(G.nodes()), "Edges (adj entries):", sum(len(G.adj[u]) for u in G.nodes()))


## 2) Layout for visualization

In [ ]:

from src.lib.adapter.layout.random_layout import RandomLayout
pos = RandomLayout(seed=42).get_positions(G)
list(pos.items())[:3]


## 3) Leiden via shared kernel (heuristic delta)

In [ ]:

from src.lib.adapter.algorithms.leiden import LeidenAdapter
from src.lib.adapter.heuristics.neighbor_weight_delta import heuristic_delta_by_neighbor_weight

algo = LeidenAdapter()  # uses SharedKernel under the hood
final_comm_heur = algo.detect(
    g=G,
    p0=None,
    delta_fn=heuristic_delta_by_neighbor_weight,
    gamma=1.0,
    theta=1.0,
    max_levels=10,
)
len(final_comm_heur), final_comm_heur


### Visualization

In [ ]:

# Each chart gets its own figure; no seaborn; do not specify colors.
import matplotlib.pyplot as plt
from src.lib.adapter.visualizer.convex_hull import MatplotlibVisualizer
MatplotlibVisualizer().render(G, final_comm_heur, pos, title="Leiden (heuristic) communities");


## 4) ObjectivePort (Modularity) + Kernel

In [ ]:

from src.lib.adapter.objectives.modularity import ModularityObjective
from src.lib.domain.services.partition import singleton_partition
from src.lib.adapter.algorithms.kernel import SharedKernel

obj_mod = ModularityObjective()
delta_mod = obj_mod.delta_fn(G, singleton_partition(G))

kernel = SharedKernel(seed=7)
p0 = singleton_partition(G)
p_end_mod = kernel.multi_level(G, p0, delta_fn=delta_mod, gamma=1.0, theta=1.0, max_levels=10)
score_mod = obj_mod.score(G, p_end_mod)
final_comm_mod = p_end_mod.flattened_partition()
print("Modularity score:", score_mod, "| # communities:", len(final_comm_mod))
final_comm_mod


## 5) Resolution scan + consensus (Leiden+)

In [ ]:

from src.lib.adapter.objectives.cpm import CPMObjective

results = algo.detect_many(
    g=G,
    p0=None,
    objectives=[ModularityObjective(), CPMObjective()],
    gammas=[0.2, 0.5, 1.0, 1.5, 2.0],
    seeds=6,
    theta_schedule=[1.0, 0.5],
    levels_per_stage=1,
)

len(results["runs"]), len(results["frontier"]), len(results["consensus"])


### Frontier table

In [ ]:

import pandas as pd
frontier_df = pd.DataFrame(results["frontier"]).sort_values(["objective", "gamma"]).reset_index(drop=True)
frontier_df


### Monotone trend (# communities vs γ) for CPM

In [ ]:

import matplotlib.pyplot as plt

pts = [pt for pt in results["frontier"] if pt["objective"] == "CPMObjective"]
xs = [pt["gamma"] for pt in pts]
ys = [pt["num_communities"] for pt in pts]

plt.figure()
plt.plot(xs, ys, marker='o')
plt.xlabel("gamma")
plt.ylabel("# communities")
plt.title("Frontier monotonicity (CPM)")
plt.show()


### Consensus summary

In [ ]:

for cons in results["consensus"]:
    print(f'{cons["objective"]} γ={cons["gamma"]} -> {len(cons["communities"])} consensus communities')
